# Capstone coursera Batalla de vecindarios - PARTE 1

 Autor: Johan valerio 2021

## Tabla de contenidos (Las 2 pregunas incluidas en el notebook)
* [Introduccion: Problema de negocio](#introduction)
* [Datos](#data)


# 1. Introduccion <a name="introduction"></a>

* La mayoria de personsa cuando quieren inciar o emprender un negocio necesitan saber acerca de la segmentacion del mercado, hacia a quienes va dirigido y mucho mejor donde hacerlo.

* Ese es el obejtivo de este pryecto de ciencia de datos,es  identificar los grupos y categorias mas prosperas en la cual inciar un negocio en Lima Peru,
como tambien saber en que lugares dicha demanda por segmentacion es mas prospera.

# PROBLEMA COMERCIAL

* Un empresario necesita saber cuales son las zonas donde tendra la certeza de obtener mayores ventas en el departamento de Lima

* Por otro lado quisiera saber cuales son las categorias de negocios que existen en dicho lugar para poder divesificar distintos productos.

# 2. SECCION DE DATOS <a name="data"></a>

* Los datos de **provincias y distritos se obtienen de Wikipedia**, especificamente de una tabla con los distritos del Peru
* De dicha relacion solo se tendra en consideracion al departamento de lima.

* Las coordenadas de datos se obtienen con la libreria **geopy** de python

* Los lugares y establecimientos con sus respecticas categorias con la **API Foursquare**


In [ ]:
import requests
import pandas as pd
url="https://es.wikipedia.org/wiki/Anexo:Distritos_del_Per%C3%BA"
data_html=requests.get(url,allow_redirects=True)

data_dirty=pd.read_html(data_html.text)

In [ ]:
df=data_dirty[0]

In [ ]:
df.columns=[ col[0].strip() for col in df.columns]

## Distritos candidatos de TABLA en wikipedia

* Mostramos la informacion de los distritos que valos a evaluar

In [ ]:
df.head()

,Ubigeo,Departamento,Provincia,Distrito,Población 2017,Población 2017
0,10101,Amazonas,Chachapoyas,Chachapoyas,32 589,NaN
1,10102,Amazonas,Chachapoyas,Asunción,262,NaN
2,10103,Amazonas,Chachapoyas,Balsas,1136,NaN
3,10104,Amazonas,Chachapoyas,Cheto,642,NaN
4,10105,Amazonas,Chachapoyas,Chiliquín,585,NaN


* Implementamos alguns tecnicas de seleccion de caracteristicas
* Para ello removemos las columnas de problacion y ubigeo
* ubigeo solo es exclusivo de algunos paises, por lo que **geopy** no implementa un busqueda por ubigeo

In [ ]:
#Filtramos la tabla para el departamento de lima
df=df[df["Departamento"]=="Lima"]
#eliminamos las columnas
df.drop(columns=["Población 2017","Ubigeo","Departamento"],axis=1,inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


* Filtramos los resultados para el departamento de lima donde haremos los estudios

In [ ]:
df.reset_index(drop=True,inplace=True)

## Coordenadas con la libreria GEOPY de python

* Necesitamos las coordenadas como punto de enlace fuerte



In [ ]:
#realizamo algnas modificaciones
df["Distrito"].replace({"Veintisiete de noviembre":"27 de noviembre"},inplace=True)
#obtendremos las coordenadasd de algunos sitios
from geopy.geocoders import Nominatim
geolocator=Nominatim(user_agent="https://www.munlima.gob.pe/")
data_coord={"latitud":[],"longitud":[]}
for district,province in zip(df["Distrito"],df["Provincia"]):
  location=geolocator.geocode("{} , {}".format(province,district))
  print(province,district)
  data_coord["latitud"].append( location.latitude if location else None)
  data_coord["longitud"].append(location.longitude if location else None)
  print(data_coord["latitud"][-1],data_coord["longitud"][-1])
data_coord=pd.DataFrame(data_coord)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Lima Lima
-12.0621065 -77.0365256
Lima Ancón
-11.69655375 -77.11165474769231
Lima Ate
-12.0387281 -76.89687298679642
Lima Barranco
-12.143959 -77.02026805535999
Lima Breña
-12.05970045 -77.05011863865556
Lima Carabayllo
-11.794993049999999 -76.98929232573605
Lima Chaclacayo
-11.9924795 -76.77617635218813
Lima Chorrillos
-12.19234965 -77.00896234092097
Lima Cieneguilla
-12.1124715 -76.8170049
Lima Comas
-11.93286125 -77.04067444714022
Lima El Agustino
-12.04205235 -76.99571439186212
Lima Independencia
-11.989307499999999 -77.04733069719748
Lima Jesús María
-12.0781861 -77.04640320962204
Lima La Molina
-12.0901768 -76.92233779667893
Lima La Victoria
-12.073993699999999 -77.01819656608845
Lima Lince
-12.08656755 -77.03664698683625
Lima Los Olivos
-11.9659853 -77.07307131998294
Lima Lurigancho
-11.948832 -76.76270115208169
Lima Lurín
-12.23804975 -76.7838627203846
Lima Magdalena del Mar
-12.0957149 -77.06824320487323
Lima Pueblo Libre
-12.07663875 -77.06785807599587
Lima Miraflores
-12.121

In [ ]:
#verificamos que todas las coordenadas se recuperaron correctamente
data_coord.isnull().any()

latitud     False
longitud    False
dtype: bool

In [ ]:
df_merge=pd.concat([df,data_coord],axis=1)

In [ ]:
df_merge.shape,df.shape,data_coord.shape

((171, 4), (171, 2), (171, 2))

In [ ]:
df_merge.head()

,Provincia,Distrito,latitud,longitud
0,Lima,Lima,-12.062107,-77.036526
1,Lima,Ancón,-11.696554,-77.111655
2,Lima,Ate,-12.038728,-76.896873
3,Lima,Barranco,-12.143959,-77.020268
4,Lima,Breña,-12.059700,-77.050119


* Vamos a visualizar el mapa de Lima

In [ ]:
#primero consultamos las coordenadas oficiales de Lima
address="Lima , Perú"
geolocator=Nominatim(user_agent="lima_explorer")
location=geolocator.geocode(address)
print(location.latitude)
latitud=location.latitude
longitud=location.longitude

-12.0621065


In [ ]:
import folium

map_lima=folium.Map(location=[latitud,longitud],zoom_start=11)
map_lima


## Uso de la API Foursquare

* Con esta obtenemos mejores recomendaciones de las categorias de negocios cercanos al lugar


In [ ]:
#agregaremos los puntos de datos en lima

for latitude,longitude,province,district in zip(df_merge["latitud"],df_merge["longitud"],df_merge["Provincia"],df_merge["Distrito"]):
  label="{0}, {1}".format(province,district)
  label=folium.Popup(label,parse_html=True)
  folium.CircleMarker(
      [latitude,longitude],
      popup=label,
      radius=5,
      color="green",
      fill=True
  ).add_to(map_lima)
map_lima

In [ ]:
CLIENT_ID="N3QXWZTKPMSVVDUMPIFCMFPFEVX0LCNQWIZIMO40YQ2AJEXE"
CLIENT_SECRET="I2DACUWQGYR12HVS3PUY5GMTCFNK2QAAIRHEYJJR11QFMBH4"
VERSION = '20180605' # Foursquare API version
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
venues_in_lima=getNearbyVenues(df_merge["Distrito"],df_merge["latitud"],df_merge["longitud"])

Lima
Ancón
Ate
Barranco
Breña
Carabayllo
Chaclacayo
Chorrillos
Cieneguilla
Comas
El Agustino
Independencia
Jesús María
La Molina
La Victoria
Lince
Los Olivos
Lurigancho
Lurín
Magdalena del Mar
Pueblo Libre
Miraflores
Pachacámac
Pucusana
Puente Piedra
Punta Hermosa
Punta Negra
Rimac
San Bartolo
San Borja
San Isidro
San Juan de Lurigancho
San Juan de Miraflores
San Luis
San Martín de Porres
San Miguel
Santa Anita
Santa María del Mar
Santa Rosa
Santiago de Surco
Surquillo
Villa El Salvador
Villa María del Triunfo
Barranca
Paramonga
Pativilca
Supe
Supe puerto
Cajatambo
Copa
Gorgor
Huancapon
Manas
Canta
Arahuay
Huamantanga
Huaros
Lachaqui
San Buenaventura
Santa Rosa de Quives
San Vicente de Cañete
Asia
Calango
Cerro azul
Chilca
Coayllo
Imperial
Lunahuana
Mala
Nuevo imperial
Pacaran
Quilmana
San Antonio
San Luis
Santa Cruz de Flores
Zuñiga
Huaral
Atavillos alto
Atavillos bajo
Aucallama
Chancay
Ihuari
Lampian
Pacaraos
San Miguel de acos
Santa Cruz de Andamarca
Sumbilca
27 de noviembre
Matucan

## GRACIAS!